In [1]:
#Install dependencies
!pip install -q transformers datasets accelerate sentencepiece huggingface_hub


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# -------------------------------
# 1. Imports
# -------------------------------
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset

In [3]:
# -------------------------------
# 2. Load Alpaca dataset
# -------------------------------
# Alpaca dataset is hosted on Hugging Face Hub
dataset = load_dataset("tatsu-lab/alpaca")

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [4]:
# Inspect one sample
print(dataset["train"][0])

{'instruction': 'Give three tips for staying healthy.', 'input': '', 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}


In [5]:
# -------------------------------
# 3. Preprocess data
# -------------------------------
model_id = "EleutherAI/gpt-neo-125M"
tokenizer = AutoTokenizer.from_pretrained(model_id)

# Fix: set pad_token to eos_token
tokenizer.pad_token = tokenizer.eos_token

def format_example(example):
    # Combine instruction + input into a single prompt
    if example.get("input"):
        prompt = f"Instruction: {example['instruction']}\nInput: {example['input']}\nOutput:"
    else:
        prompt = f"Instruction: {example['instruction']}\nOutput:"
    return {"text": prompt + example["output"]}

formatted_dataset = dataset["train"].map(format_example)

def tokenize(batch):
    return tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)

tokenized_dataset = formatted_dataset.map(tokenize, batched=True)

tokenizer_config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [6]:
# -------------------------------
# 4. Load base model
# -------------------------------
model = AutoModelForCausalLM.from_pretrained(model_id)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/526M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [14]:
!pip install -U transformers accelerate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 45.8 MB/s eta 0:00:00 0:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.1
    Uninstalling transformers-4.57.1:
      Successfully uninstalled transformers-4.57.1

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [9]:
import torch
print(torch.backends.mps.is_available())   # True if MPS is supported
print(torch.backends.mps.is_built())       # True if PyTorch was built with MPS

True
True


In [11]:
device = torch.device("mps")

In [12]:
# -------------------------------
# 5. Training setup
# -------------------------------
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=2,   # small batch for MPS
    per_device_eval_batch_size=2,
    num_train_epochs=3,
    learning_rate=5e-5,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
    warmup_steps=100,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(1000)),
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/var/folders/_s/sj_9z5414nj8xvj6fqhvy9380000gn/T/ipykernel_24751/2813590258.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:

# -------------------------------
# 6. Train
# -------------------------------
trainer.train()


/Users/satyanarayanaallam/Projects/.venv/lib/python3.14/site-packages/torch/utils/data/dataloader.py:692: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
100,2.520900
200,2.321100
300,2.282300
400,2.313400
500,2.328000
600,2.294900
700,2.236900
800,2.341700
900,2.233000
1000,2.236300


In [ ]:
#-------------------------------
# 7. Save model
# -------------------------------
trainer.save_model("./alpaca-gptneo-125m")
tokenizer.save_pretrained("./alpaca-gptneo-125m")

# -------------------------------
# 8. Test inference
# -------------------------------
from transformers import pipeline

pipe = pipeline("text-generation", model="./alpaca-gptneo-125m", tokenizer=tokenizer)

print(pipe("Instruction: Write a poem about F1 racing\nOutput:", max_new_tokens=100)[0]["generated_text"])